In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import json
import pandas as pd

train_file_path = '/content/drive/My Drive/small-PhoMT/small-train.json'
dev_file_path = '/content/drive/My Drive/small-PhoMT/small-dev.json'
test_file_path = '/content/drive/My Drive/small-PhoMT/small-test.json'

# Mở và đọc file JSON
with open(train_file_path, 'r', encoding='utf-8') as f:
    train_data = pd.read_json(f)

with open(dev_file_path, 'r', encoding='utf-8') as f:
    dev_data = pd.read_json(f)

with open(test_file_path, 'r', encoding='utf-8') as f:
    test_data = pd.read_json(f)

# Bây giờ bạn có thể bắt đầu tiền xử lý hoặc huấn luyện mô hình
print(f"Số lượng bản ghi trong train data: {len(train_data)}")
print(f"Số lượng bản ghi trong valid data: {len(dev_data)}")
print(f"Số lượng bản ghi trong test data: {len(test_data)}")

Số lượng bản ghi trong train data: 20000
Số lượng bản ghi trong valid data: 2000
Số lượng bản ghi trong test data: 2000


In [5]:
!pip install torch==2.3.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 124.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 837.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1

In [6]:
!pip install torchtext==0.18.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.4 MB/s eta 0:00:00


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
import pandas as pd
from transformers import AutoTokenizer
import numpy as np
import random
import math
from tqdm.notebook import tqdm

# Đặt seed để đảm bảo kết quả có thể lặp lại
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/usr/local/lib/python3.12/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [8]:
tokenizer_vi = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=False)

# Sử dụng Tokenizer cơ bản của Hugging Face cho tiếng Anh (Rất ổn định)
tokenizer_en = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=False)

# Sửa đổi quan trọng: Thêm thuộc tính clean-up
def clean_tokenizer_output(tokens, tokenizer):
    """
    Loại bỏ các token đặc biệt của tokenizer (như [CLS], [SEP], <s>, </s>)
    ra khỏi chuỗi output.
    """
    special_tokens = tokenizer.all_special_tokens
    # Lọc các token đặc biệt ra khỏi output
    return [t for t in tokens if t not in special_tokens]

def tokenize_en(text):
    """Tokenizes tiếng Anh bằng BERT Tokenizer và loại bỏ token đặc biệt."""
    tokens = tokenizer_en.tokenize(text.lower())
    return clean_tokenizer_output(tokens, tokenizer_en) # Lọc token đặc biệt của BERT

def tokenize_vi(text):
    """Tokenizes tiếng Việt bằng PhoBERT Tokenizer và loại bỏ token đặc biệt."""
    tokens = tokenizer_vi.tokenize(text.lower())

    # Lọc token đặc biệt của PhoBERT (<s>, </s>)
    cleaned_tokens = clean_tokenizer_output(tokens, tokenizer_vi)

    # Thêm token <sos> và <eos> đã được định nghĩa trong special_symbols của bạn
    return ['<sos>'] + cleaned_tokens + ['<eos>']

# Xây dựng Vocab
def yield_tokens(data_iter, language):
    """Hàm yield tokens từ DataFrame"""
    for _, row in data_iter.iterrows():
        if language == 'en':
            yield tokenize_en(row['english'])
        elif language == 'vi':
            yield tokenize_vi(row['vietnamese'])

# Các token đặc biệt
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

# Xây dựng Vocab
vocab_src = build_vocab_from_iterator(
    yield_tokens(train_data, 'en'),
    min_freq=2, # Chỉ giữ lại các từ xuất hiện ít nhất 2 lần
    specials=special_symbols
)
vocab_trg = build_vocab_from_iterator(
    yield_tokens(train_data, 'vi'),
    min_freq=2,
    specials=special_symbols
)

# Tự động gán index UNK_IDX cho những từ nằm ngoài vocab.
vocab_src.set_default_index(UNK_IDX)
vocab_trg.set_default_index(UNK_IDX)

# Kích thước từ vựng
INPUT_DIM = len(vocab_src)
OUTPUT_DIM = len(vocab_trg)
print(f"Kích thước từ vựng tiếng Anh (Input): {INPUT_DIM}")
print(f"Kích thước từ vựng tiếng Việt (Output): {OUTPUT_DIM}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Kích thước từ vựng tiếng Anh (Input): 10699
Kích thước từ vựng tiếng Việt (Output): 4699


In [9]:
# Biến dữ liệu DataFrame thành một bộ sưu tập các cặp Tensor đã được chỉ số hóa, sẵn sàng được nạp vào mô hình.
class TranslationDataset(Dataset):
    def __init__(self, df, vocab_src, vocab_trg):
        self.df = df
        self.vocab_src = vocab_src
        self.vocab_trg = vocab_trg

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        en_text = self.df.iloc[idx]['english']
        vi_text = self.df.iloc[idx]['vietnamese']

        # Tách token
        en_tokens = tokenize_en(en_text)
        vi_tokens = tokenize_vi(vi_text)

        # Chuyển token sang index
        en_indices = self.vocab_src.lookup_indices(en_tokens)
        vi_indices = self.vocab_trg.lookup_indices(vi_tokens)

        return torch.LongTensor(en_indices), torch.LongTensor(vi_indices)

# Collate function để padding các chuỗi trong cùng một batch
def collate_fn(batch):
    src_batch, trg_batch = [], []
    for src, trg in batch:
        src_batch.append(src)
        trg_batch.append(trg)

    # Padding: Hàm pad_sequence sẽ mặc định padding đến độ dài lớn nhất của sample trong batch
    src_batch = nn.utils.rnn.pad_sequence(src_batch, padding_value=PAD_IDX)
    trg_batch = nn.utils.rnn.pad_sequence(trg_batch, padding_value=PAD_IDX)

    return src_batch, trg_batch

# Tạo Dataset và DataLoader
BATCH_SIZE = 64
train_dataset = TranslationDataset(train_data, vocab_src, vocab_trg)
dev_dataset = TranslationDataset(dev_data, vocab_src, vocab_trg)
test_dataset = TranslationDataset(test_data, vocab_src, vocab_trg)

train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
dev_iterator = DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [10]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        # Theo bài báo, năng lượng e_ij = v_a^T * tanh(W_a * s_{i-1} + U_a * h_j)
        # enc_hid_dim * 2 vì Encoder là Bidirectional (theo đề xuất của Bahdanau
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)

    def forward(self, hidden, encoder_outputs):
        # hidden = [batch size, dec hid dim] (Trạng thái ẩn s_{i-1} của Decoder)
        # encoder_outputs = [src len, batch size, enc hid dim * 2] (Các annotation h_j)

        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        # Lặp lại hidden state src_len lần để khớp kích thước với encoder_outputs
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # hidden = [batch size, src len, dec hid dim]
        # encoder_outputs = [batch size, src len, enc hid dim * 2]

        # Tính năng lượng (Energy)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2)))
        # energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        # attention = [batch size, src len]

        # Trả về softmax để có tổng trọng số = 1 [cite: 104]
        return F.softmax(attention, dim=1)

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)

        # Encoder dùng Bidirectional theo paper [cite: 74]
        self.rnn = nn.LSTM(emb_dim, enc_hid_dim, num_layers=3, bidirectional=True, dropout=dropout)

        # Fully connected layer để chuyển đổi hidden state của Encoder (2 chiều)
        # sang hidden state ban đầu của Decoder (1 chiều)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))

        # outputs = [src len, batch size, enc hid dim * 2]
        # hidden = [n layers * 2, batch size, enc hid dim]
        # cell = [n layers * 2, batch size, enc hid dim]
        outputs, (hidden, cell) = self.rnn(embedded)

        # Vì Encoder là 2 chiều + 3 lớp, còn Decoder là 1 chiều + 3 lớp,
        # Ta cần xử lý hidden state và cell state để khởi tạo cho Decoder.

        # Lấy hidden/cell state của lớp cuối cùng (cả forward và backward)
        # hidden[-2,:,:] là forward lớp cuối, hidden[-1,:,:] là backward lớp cuối
        # hidden_last được dùng để khởi tạo hidden state của Decoder
        hidden_last = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        # Chuẩn bị cell state cho 3 lớp LSTM của Decoder
        # Decoder có 3 lớp unidirectional, ta lấy cell state của 3 lớp forward của Encoder
        # cell_decoder = [3, batch size, dec_hid_dim]
        # Vì cell[-2] và cell[-1] là forward/backward của lớp cuối, ta cần xử lý tương tự
        cell_last = self.fc(torch.cat((cell[-2,:,:], cell[-1,:,:]), dim = 1)).unsqueeze(0)  # [1, batch, dec_hid]
        cell_decoder = cell_last.repeat(3, 1, 1)  # [3, batch, dec_hid]

        # Trả về outputs (để tính Attention), hidden_last (s_0 của Decoder), và cell_decoder
        return outputs, hidden_last, cell_decoder

In [12]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        # Input của LSTM bây giờ là: Embedding + Context Vector
        self.rnn = nn.LSTM((enc_hid_dim * 2) + emb_dim, dec_hid_dim, num_layers=3, dropout=dropout)

        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs, cell):
        # input = [batch size] (1 từ tại 1 thời điểm)
        # hidden = [batch size, dec hid dim] (s_{i-1} - hidden state từ bước trước)
        # encoder_outputs = [src len, batch size, enc hid dim * 2]
        # cell = [n_layers, batch size, dec hid dim]

        input = input.unsqueeze(0) # [1, batch size]
        embedded = self.dropout(self.embedding(input)) # [1, batch size, emb dim]

        # BAHDANAU ATTENTION WORKFLOW:
        # 1. Tính Attention weights a_t dùng s_{i-1} (hidden state từ bước trước)
        # Công thức: e_ij = v_a^T * tanh(W_a * s_{i-1} + U_a * h_j)
        # α_ij = softmax(e_ij)
        a = self.attention(hidden, encoder_outputs) # [batch size, src len]
        a = a.unsqueeze(1) # [batch size, 1, src len]

        # 2. Tính Context Vector c_i = Σ α_ij * h_j
        encoder_outputs_perm = encoder_outputs.permute(1, 0, 2) # [batch size, src len, enc hid dim * 2]
        weighted = torch.bmm(a, encoder_outputs_perm) # [batch size, 1, enc hid dim * 2]
        weighted = weighted.permute(1, 0, 2) # [1, batch size, enc hid dim * 2]

        # 3. Nối Input Embedding và Context Vector: [y_{i-1}, c_i]
        rnn_input = torch.cat((embedded, weighted), dim = 2) # [1, batch size, emb_dim + enc_hid_dim*2]

        # 4. Bước RNN: s_i = LSTM(s_{i-1}, [y_{i-1}, c_i])
        # Lưu ý: Attention được tính từ s_{i-1}, KHÔNG phải s_i
        output, (hidden_new, cell_new) = self.rnn(rnn_input, (hidden.unsqueeze(0).repeat(3, 1, 1), cell))

        # output = [1, batch size, dec hid dim]
        # hidden_new = [n_layers, batch size, dec hid dim]
        # cell_new = [n_layers, batch size, dec hid dim]

        # 5. Dự đoán từ tiếp theo: Σ = tanh(W_c[c_i, s_i])
        #    p(y_i) = softmax(W_s * Σ) [cite: Bahdanau]
        output_squeezed = output.squeeze(0)  # [batch size, dec hid dim]
        weighted_squeezed = weighted.squeeze(0)  # [batch size, enc hid dim * 2]
        embedded_squeezed = embedded.squeeze(0)  # [batch size, emb dim]

        # Concatenate: [s_i, c_i, y_{i-1}]
        prediction = self.fc_out(torch.cat((output_squeezed, weighted_squeezed, embedded_squeezed), dim = 1))

        # Trả về hidden[-1] (layer cuối) để dùng trong bước tiếp theo
        # Lần tới, hidden này sẽ trở thành s_{i-1} để tính attention
        return prediction, hidden_new[-1], cell_new

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        # src = [src len, batch size]
        # trg = [trg len, batch size]

        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # Encoder trả về tất cả hidden states (cho attention), hidden state cuối cùng (để init decoder)
        # và cell state cho 3 lớp của Decoder
        encoder_outputs, hidden, cell = self.encoder(src)

        # hidden = [batch size, dec hid dim] (s_0 của Decoder)
        # cell = [3, batch size, dec hid dim]

        input = trg[0,:]

        for t in range(1, trg_len):

            # Decoder nhận thêm encoder_outputs
            output, hidden, cell = self.decoder(input, hidden, encoder_outputs, cell)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

In [14]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
# Cấu hình Hyperparameters
INPUT_DIM = len(vocab_src)
OUTPUT_DIM = len(vocab_trg)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# Khởi tạo
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, DEVICE).to(DEVICE)

# Khởi tạo trọng số
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

print(f'Mô hình có {sum(p.numel() for p in model.parameters() if p.requires_grad):,} tham số có thể huấn luyện')

Mô hình có 15,396,699 tham số có thể huấn luyện


In [16]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()
    epoch_loss = 0

    for i, (src, trg) in enumerate(tqdm(iterator, desc="Training")):

        src = src.to(DEVICE)
        trg = trg.to(DEVICE)

        optimizer.zero_grad()

        # outputs = [trg_len, batch_size, trg_vocab_size]
        output = model(src, trg)

        # trg = [trg_len, batch_size]
        # Cần flatten output và trg để tính Loss
        # Skip token <sos> ở index 0
        output_dim = output.shape[-1]

        # output_flat = [(trg_len - 1) * batch_size, output_dim]
        output = output[1:].view(-1, output_dim)

        # trg_flat = [(trg_len - 1) * batch_size]
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        # Cắt gradient để tránh hiện tượng exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):

    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, (src, trg) in enumerate(tqdm(iterator, desc="Evaluating")):

            src = src.to(DEVICE)
            trg = trg.to(DEVICE)

            # Tắt teacher forcing (ratio = 0)
            output = model(src, trg, 0) #turn off teacher forcing

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [17]:
import torch
import gc

# Xóa cache của PyTorch
torch.cuda.empty_cache()

# Thu gom rác của Python
gc.collect()

0

In [18]:
N_EPOCHS = 20
CLIP = 1
best_dev_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    dev_loss = evaluate(model, dev_iterator, criterion)

    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {dev_loss:.3f} | Val. PPL: {math.exp(dev_loss):.3f}')

Training:   0%|          | 0/313 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 01 | Train Loss: 6.118 | Val. Loss: 6.384 | Val. PPL: 592.069


Training:   0%|          | 0/313 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 19 | Train Loss: 3.204 | Val. Loss: 5.887 | Val. PPL: 360.201


Training:   0%|          | 0/313 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 20 | Train Loss: 3.137 | Val. Loss: 5.927 | Val. PPL: 375.177


In [19]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ee24a89e86a5ac5ebda2d5a5ff453bf93118a8c4e427f2d9f2baa8792e4d9547
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [20]:
from rouge_score import rouge_scorer


# Tải mô hình tốt nhất
model.load_state_dict(torch.load('tut2-model.pt'))

def translate_sentence(sentence, src_vocab, trg_vocab, model, device, max_len = 50):
    model.eval()

    # 1. Tokenize và Index
    tokens = tokenize_en(sentence)
    src_indexes = src_vocab.lookup_indices(tokens)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 2. Encoding
    with torch.no_grad():
        # Encoder trả về: encoder_outputs (cho attention), hidden (s_0), cell (khởi tạo cho Decoder)
        encoder_outputs, hidden, cell = model.encoder(src_tensor)

    # hidden = [batch_size=1, dec_hid_dim]
    # cell = [n_layers=3, batch_size=1, dec_hid_dim]

    # 3. Decoding
    trg_indexes = [trg_vocab['<sos>']]

    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            # Decoder nhận: input, hidden (s_{i-1}), encoder_outputs, cell
            output, hidden, cell = model.decoder(trg_tensor, hidden, encoder_outputs, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        if pred_token == trg_vocab['<eos>']:
            break

    # 4. Chuyển Index thành Text
    trg_tokens = trg_vocab.lookup_tokens(trg_indexes)

    return trg_tokens[1:] # Bỏ <sos> # Bỏ <sos>

def calculate_rouge_l(model, iterator, src_vocab, trg_vocab, device):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    total_rouge_l = 0
    count = 0

    for src_batch, trg_batch in tqdm(iterator, desc="Calculating ROUGE-L"):
        # Trg (ground truth)
        trg_batch = trg_batch.transpose(0, 1) # [batch_size, trg_len]

        for i in range(trg_batch.shape[0]):
            # Chuyển sentence source tensor sang text
            src_tensor = src_batch[:, i].unsqueeze(1)
            src_indices = src_tensor.squeeze(1).tolist()
            # 1. Loại bỏ <pad> và <unk> khỏi câu nguồn trước khi dịch
            src_tokens_raw = src_vocab.lookup_tokens(src_indices)
            src_tokens = [t for t in src_tokens_raw if t not in ['<pad>', '<unk>']]
            src_sentence = ' '.join(src_tokens)

            # Dịch sentence
            hyp_tokens = translate_sentence(src_sentence, src_vocab, trg_vocab, model, device)
            # 2. Xử lý hypothesis (câu dự đoán): Loại bỏ <unk> và các token thừa
            hypothesis = ' '.join([t for t in hyp_tokens if t not in ['<unk>', '<eos>', '<pad>']]).strip()

            # Ground truth
            ref_tokens_raw = trg_vocab.lookup_tokens(trg_batch[i].tolist())
            # 3. Xử lý reference (câu tham chiếu): Loại bỏ tất cả các token đặc biệt
            reference = ' '.join([t for t in ref_tokens_raw if t not in ['<sos>', '<eos>', '<pad>', '<unk>']]).strip()

            if not reference or not hypothesis:
                continue

            score = scorer.score(reference, hypothesis)
            total_rouge_l += score['rougeL'].fmeasure
            count += 1

    return total_rouge_l / count if count > 0 else 0

rouge_l_score = calculate_rouge_l(model, test_iterator, vocab_src, vocab_trg, DEVICE)
print(f"\n Đánh giá ROUGE-L trên tập Test: {rouge_l_score:.4f}")

Calculating ROUGE-L:   0%|          | 0/32 [00:00<?, ?it/s]


 Đánh giá ROUGE-L trên tập Test: 0.3742
